In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Tue Dec  1 14:39:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!git clone https://github.com/adapter-hub/adapter-transformers.git
%cd adapter-transformers

Cloning into 'adapter-transformers'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 42519 (delta 10), reused 8 (delta 2), pack-reused 42490
Receiving objects: 100% (42519/42519), 23.88 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (30833/30833), done.
/content/adapter-transformers


In [5]:
!pip install .
!pip install sklearn

Processing /content/adapter-transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 5.9MB/s 
     |████████████████████████████████| 2.9MB 31.6MB/s 
     |████████████████████████████████| 890kB 42.4MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-1.1.0-cp36-none-any.whl size=1325191 sha256=6a947231c979cdfb6eea34ed6623eac3d7b219c917d4fa7b33688593132309d8
  Stored in directory: /root/.cache/pip/wheels/69/f2/bd/f0720d145cbd02c42f9d833e06bb319e9ee518b24ebaa092e1
Successfully built adapter-transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8c1e52ae7e7852a20defa60cb4b1df444731f35b12a0b5d18e536fd784e2ff20
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
%cd ../temp/adapticons
%mkdir datasets
%cd datasets/
%mkdir hyper
%cd hyper

/content/temp/adapticons
/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/hyper


In [10]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/hyperpartisan_news/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/hyperpartisan_news/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/hyperpartisan_news/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1708k  100 1708k    0     0  1885k      0 --:--:-- --:--:-- --:--:-- 1883k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k  100  197k    0     0   324k      0 --:--:-- --:--:-- --:--:--  324k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241k  100  241k    0     0   337k      0 --:--:-- --:--:-- --:--:--  337k


In [11]:
%cd ../../modeling

/content/temp/adapticons/modeling


In [12]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/hyper/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/tapt_hyper/newtrain/ \
  --task_name HYPER \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/tapt_hyper/ \
  --metric macro \

2020-12-01 14:41:21.082192: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/01/2020 14:41:23 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/01/2020 14:41:23 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/tapt_hyper/newtrain/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, 

In [13]:
%cd ../datasets/
%mkdir citation
%cd citation/

/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/citation


In [14]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0   498k      0 --:--:-- --:--:-- --:--:--  498k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30138  100 30138    0     0  67122      0 --:--:-- --:--:-- --:--:-- 66973
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38637  100 38637    0     0  82381      0 --:--:-- --:--:-- --:--:-- 82381


In [15]:
%cd ../../modeling

/content/temp/adapticons/modeling


In [16]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/tapt_citation/newtrain/ \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/tapt_citation/ \
  --metric macro \

2020-12-01 14:51:31.903534: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/01/2020 14:51:33 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/01/2020 14:51:33 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/tapt_citation/newtrain/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=

In [17]:
%cd ../datasets/
%mkdir sciie
%cd sciie/

/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/sciie


In [18]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  804k  100  804k    0     0   844k      0 --:--:-- --:--:-- --:--:--  843k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  115k  100  115k    0     0   171k      0 --:--:-- --:--:-- --:--:--  170k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  243k  100  243k    0     0   325k      0 --:--:-- --:--:-- --:--:--  325k


In [19]:
%cd ../../modeling

/content/temp/adapticons/modeling


In [20]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/sciie/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/tapt_sciie/newtrain/ \
  --task_name sciie \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/tapt_sciie/ \
  --metric macro \

2020-12-01 15:11:16.453622: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/01/2020 15:11:18 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/01/2020 15:11:18 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/tapt_sciie/newtrain/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, 